In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import spatial

import os

In [ ]:
from shapely.geometry import Point
from shapely.geometry import Polygon

In [ ]:
plt.style.use('Solarize_Light2')   # try another styles: 'classic'

In [ ]:
#driver = "~/"  # "G:/
driver = "G:/"
destData = os.path.join(driver, "1_Data1/sea/AllSeaGisOut.txt")
batimetrFile = "G:/1_Data1/batimetria/b1.txt"

In [ ]:
obris = pd.read_csv(os.path.join(driver, "1_Data1/obris/obris.dat"),
                    header=0,
                    na_values='--',
                    sep='\t',
                    decimal=',',
                    nrows=5425
                   )
obris.loc[np.isnan(obris['lat']),:]

In [ ]:
obris_olkhon = obris.loc[4845:5298,:]
obris_olkhon

In [ ]:
plt.scatter('long', 'lat', data=obris, s=1)
plt.scatter('long', 'lat', data=obris_olkhon, s=1)

In [ ]:
obrisEast = pd.read_csv(os.path.join(driver, "1_Data1/obris/obrisEast.dat"),
                        header=0,
                        na_values='--',
                        sep='\t',
                        decimal=',',
                       )

In [ ]:
obrisWest = pd.read_csv(os.path.join(driver, "1_Data1/obris/obrisWest.dat"),
                        header=0,
                        na_values='--',
                        sep='\t',
                        decimal=',',
                       )
obrisWest.head(3)

In [ ]:
plt.rcParams['figure.figsize'] = [4, 5]
fig, ax = plt.subplots()
ax.plot(obrisEast['long'], obrisEast['lat'], 'b-', alpha=0.4)
ax.plot(obrisWest['long'], obrisWest['lat'], 'r-', alpha=0.4)


## GIS handlers   
### Get Squared Baikal

In [ ]:
def dist(a, b):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees,
    [0] - long,
    [1] - lat)
    """
    x1, y1, x2, y2 = map(np.radians, [a[0], a[1], b[0], b[1]])
    dx = x2 - x1
    dy = y2 - y1
    a = np.sin(dy/2)**2 + np.cos(y1) * np.cos(y2) * np.sin(dx/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    # Earth`s radius in kilometers
    km = 6372.795 * c
    return km


def rdist(a, b):
    """
    Calculate the great circle distance between two points
    on the sphere, radius 1 ru (specified in radians, x1, y1, x2, y2
    x - long, y - lat).
    multiply result to Earth`s dia (6372.795 km) to get metric distance on Earth surface
    """
    x1, y1 = a
    x2, y2 =b
    return 2 * np.arcsin(np.sqrt(np.sin((y2 - y1)/2)**2 + np.cos(y1) * np.cos(y2) * np.sin((x2 - x1)/2)**2))

In [ ]:
tomsk = (85.1974, 56.372)
moscow = (37.929, 55.422)
a = (85, 51)
b = (85, 51.01)
dist(a, b)

In [ ]:
nordPoint = (109.767, 55.751)
sudPoint = (103.70917, 51.707917)

In [ ]:
obrisWest['longr'] = np.radians(obrisWest['long'])
obrisWest['latr'] = np.radians(obrisWest['lat'])
obrisEast['longr'] = np.radians(obrisEast['long'])
obrisEast['latr'] = np.radians(obrisEast['lat'])

In [ ]:
ob = zip(list(obrisWest['long'])+(list(obrisEast['long'])) , list(obrisWest['lat'])+(list(obrisEast['lat'])))

In [ ]:
obrisPolygon = Polygon(obris_olkhon)
obrisPolygon

In [ ]:
dff = 0
dff = pd.DataFrame(columns=['long', 'lat'])

n=0
for x in range(10350, 11000, round(longStep*100)):
    print(f'long = {x/100}; ', end='')
    for y in np.arange(5140, 5590, round(latStep*100)):
        if obrisPolygon.contains(Point(x/100, y/100)):
            dff.loc[len(dff), ['long', 'lat']] = [x/100, y/100]
            n+=1
            if n%100 == 0:
                print(f"n={n},", end=' ')
print("done!")

In [ ]:
longStep = 0.02
latStep = 0.01

In [ ]:
dist((0, 51.5), (longStep, 51.5)),  dist((0, 55.9), (longStep, 55.9))

In [ ]:
dist((110, 51.5), (110, 51.5+latStep))

In [ ]:
dff = 0
dff = pd.DataFrame(columns=['long', 'lat'])

n=0
for x in range(10350, 11000, round(longStep*100)):
    print(f'long = {x/100}; ', end='')
    for y in np.arange(5140, 5590, round(latStep*100)):
        if obrisPolygon.contains(Point(x/100, y/100)):
            toWest = obrisWest.apply(lambda f: dist((f['long'], f['lat']), (x/100, y/100)), axis=1).min()
            toEast = obrisEast.apply(lambda f: dist((f['long'], f['lat']), (x/100, y/100)), axis=1).min()
            toSud = dist(sudPoint, (x/100, y/100))
            toNord = dist(nordPoint, (x/100, y/100))
            dff.loc[len(dff), ['long', 'lat', 'toWest', 'toEast', 'toSud', 'toNord']] = [x/100, y/100, toWest, toEast, toSud, toNord]
            n+=1
            if n%100 == 0:
                print(f"n={n},", end=' ')
print("done!")

In [ ]:
dff.loc[::-1,['lat']]

In [ ]:
dff.to_csv('wholeBaikalGridolkhon_.dat',
           sep='\t')

In [ ]:
dff

In [ ]:
plt.rcParams['figure.figsize'] = [15, 18]
plt.scatter('long', 'lat', data=dff, s=1, alpha=0.5)

In [ ]:
plt.scatter('norm_x', 'norm_xx', data=res, s=5, alpha=0.2)

In [ ]:
# This solution by @denis is OK for small xy2
res = pd.DataFrame(columns=['long', 'lat'])

res['mindistWest'] = np.min(spatial.distance.cdist(res.loc[:, ['rlong', 'rlat']], obrisWest.loc[:, ['rlong', 'rlat']], metric=rdist), axis=1)
res['mindistEast'] = np.min(spatial.distance.cdist(res.loc[:, ['rlong', 'rlat']], obrisEast.loc[:, ['rlong', 'rlat']], metric=rdist), axis=1)
res['norm_x'] = res['mindistWest']/(res['mindistWest'] + res['mindistEast'])
res['toNord'] = np.min(spatial.distance.pdist(res.loc[:, ['rlong', 'rlat']], obrisWest.loc[:, ['rlong', 'rlat']], metric=rdist), axis=1)
res['mindistEast'] = np.min(spatial.distance.cdist(res.loc[:, ['rlong', 'rlat']], obrisEast.loc[:, ['rlong', 'rlat']], metric=rdist), axis=1)

res['norm_y'] = res['toSud']/(res['toSud'] + res['toNord'])

res

In [ ]:
l = list(obrisWest['long']) + list(obrisEast['long'])

In [ ]:
point = Point(103, 55)

In [ ]:
obrisPolygon.contains(point)

In [ ]:
dff = pd.DataFrame(columns=['long', 'lat'])
dff

#### fill baikal obris with cells `longStep` * `latStep`  
do it once. do no forget save to file

In [ ]:
dff['norm_x'] = np.int32(dff['toWest']/(dff['toWest'] + dff['toEast'])*100)
dff['norm_y'] = np.int32(dff['toSud']/(dff['toSud'] + dff['toNord'])*100)
dff

In [ ]:
dff.plot('long', 'lat', c=dff['norm_x'], kind='scatter',)